This script loads behavioral mice data for specified mice. The data are slightly reformatted and saved as `.csv` files.

In [1]:
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference, data
from ibl_pipeline.analyses.behavior import PsychResults, SessionTrainingStatus
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.analyses import behavior as behavior_analysis

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Connecting lacerbi@datajoint.internationalbrainlab.org:3306


In [3]:
import os
myPath = r"C:\Users\Luigi\Documents\GitHub\ibl-changepoint\data" # Write here your data path
os.chdir(myPath)

In [4]:
# Get list of mice that satisfy given training criteria (stringent trained_1b)
# Check query from behavioral paper: 
# https://github.com/int-brain-lab/paper-behavior/blob/master/paper_behavior_functions.py

mice_names = ['ZM_1091','CSHL_003']

print(mice_names)

['ZM_1091', 'CSHL_003']


In [5]:
sess_stable = (acquisition.Session * behavior_analysis.SessionTrainingStatus)
#               & 'task_protocol LIKE "%biased%"' & 'session_start_time < "2019-09-30"')
              
stable_mice_names = list()

def get_mouse_data(df):
    position_deg = 35.    # Stimuli appear at +/- 35 degrees

    # Create new dataframe
    datamat = pd.DataFrame()
    datamat['trial_num'] = df['trial_id']
    datamat['session_num'] = np.cumsum(df['trial_id'] == 1)
    datamat['stim_probability_left'] = df['trial_stim_prob_left']
    signed_contrast = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']
    datamat['contrast'] = np.abs(signed_contrast)
    datamat['position'] = np.sign(signed_contrast)*position_deg
    datamat['response_choice'] = df['trial_response_choice']
    datamat.loc[df['trial_response_choice'] == 'CCW','response_choice'] = 1
    datamat.loc[df['trial_response_choice'] == 'CW','response_choice'] = -1
    datamat.loc[df['trial_response_choice'] == 'No Go','response_choice'] = 0
    datamat['trial_correct'] = np.double(df['trial_feedback_type']==1)
    datamat['reaction_time'] = df['trial_response_time'] - df['trial_stim_on_time'] # double-check
    date2str = lambda x: int(str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
    datamat['date'] = df['session_start_time'].map(date2str)

    # Since some trials have zero contrast, need to compute the alleged position separately
    datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'response_choice']*position_deg
    datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'response_choice']*(-position_deg)
    
    return datamat

In [6]:
# Loop over all mice
for mouse_nickname in mice_names:
    
    mouse_subject = {'subject_nickname': mouse_nickname}
    
    # Get mouse data for biased sessions
    behavior_stable = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
        * sess_stable.proj('session_uuid','task_protocol','session_start_time','training_status') * subject.Subject.proj('subject_nickname') \
        * subject.SubjectLab.proj('lab_name')
    df = pd.DataFrame(behavior_stable.fetch(order_by='subject_nickname, session_start_time, trial_id', as_dict=True))

    datamat = get_mouse_data(df)

    # Save dataframe to CSV file
    filename = mouse_nickname + '.csv'
    datamat.to_csv(filename,index=False)
    stable_mice_names.append(mouse_nickname)
                        
print(stable_mice_names)

['ZM_1091', 'CSHL_003']
